In [1]:
import warnings
import pprint
import skrebate
import imblearn
from imblearn import under_sampling, over_sampling, combine
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn import (preprocessing, svm, linear_model, ensemble, naive_bayes,
                    tree, neighbors, decomposition, kernel_approximation, cluster)
from sklearn.pipeline import Pipeline

from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import KFold, GroupKFold, cross_validate, cross_val_predict
from sklearn.feature_selection import SelectKBest, f_regression, SelectFromModel, VarianceThreshold
from sklearn.metrics import r2_score, auc, roc_auc_score, balanced_accuracy_score
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from xgboost import XGBRegressor

warnings.simplefilter('ignore')



/Users/guq/galaxy/.venv/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/Users/guq/galaxy/.venv/lib/python2.7/site-packages/matplotlib/mpl-data/matplotlibrc", line #620
  (fname, cnt))


In [2]:
import numpy as np
import pandas as pd
import re


import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [4]:
# test3 master dataset
test3_master = pd.read_csv('./mIHC/test3/test3_masterset.csv', sep=',', index_col=0)
test3_master

,Intensity_IntegratedIntensityEdge_CD138_rs,Intensity_IntegratedIntensityEdge_CD20_rs,Intensity_IntegratedIntensityEdge_CD27_rs,Intensity_IntegratedIntensityEdge_CD38_rs,Intensity_IntegratedIntensityEdge_CD3_rs,Intensity_IntegratedIntensityEdge_CD45_rs,Intensity_IntegratedIntensityEdge_CD4_rs,Intensity_IntegratedIntensityEdge_CD5_rs,Intensity_IntegratedIntensityEdge_CD68_rs,Intensity_IntegratedIntensityEdge_CD8_rs,...,Intensity_UpperQuartileIntensity_PANCK_rs,Intensity_UpperQuartileIntensity_PD1_rs,Intensity_UpperQuartileIntensity_PDL1_rs,Intensity_UpperQuartileIntensity_TBET_rs,Intensity_UpperQuartileIntensity_TCRB_rs,Location_Center_X,Location_Center_Y,obj_ID,AreaShape_Area,class
1,0.008000,0.000000,0.025000,0.000000,0.018182,0.018349,0.272727,0.015385,0.000000,0.012270,...,0.007353,0.017857,0.000000,0.000000,0.000000,24.500000,1.285714,SMA06F_1,14,G
2,0.120000,0.015385,1.808333,0.255682,4.818182,6.128440,4.383838,3.374359,0.004695,0.184049,...,0.000000,0.044643,0.036585,0.034188,0.004608,36.521739,3.144928,SMA06F_2,69,B
3,0.376000,0.569231,9.591667,0.187500,6.642424,10.055046,5.979798,2.651282,0.046948,0.190184,...,0.004902,0.473214,0.012195,0.094017,0.004608,110.648148,5.055556,SMA06F_3,108,B
4,0.064000,0.023077,0.258333,0.096591,4.369697,3.587156,1.323232,1.266667,0.009390,0.024540,...,0.000000,0.008929,0.000000,0.034188,0.000000,130.466667,3.746667,SMA06F_4,75,B
5,0.168000,0.030769,0.433333,0.011364,6.466667,2.229358,3.181818,8.225641,0.436620,0.006135,...,0.004902,0.102679,0.018293,0.038462,0.200461,149.354545,6.881818,SMA06F_5,110,B
6,0.584000,0.315385,0.458333,0.017045,0.284848,0.275229,1.191919,0.333333,0.000000,0.184049,...,0.186275,0.017857,0.048780,0.000000,0.004608,310.659574,2.851064,SMA06F_6,94,H
7,1.864000,0.015385,0.325000,0.017045,0.181818,0.137615,1.434343,0.076923,0.009390,0.061350,...,0.246324,0.035714,0.024390,0.000000,0.000000,343.705263,6.473684,SMA06F_7,95,H
8,0.128000,0.015385,0.183333,0.068182,1.557576,2.073395,2.727273,0.230769,2.788732,0.036810,...,0.009804,0.035714,0.036585,0.000000,0.000000,378.138889,5.097222,SMA06F_8,72,D
9,0.552000,0.000000,0.058333,0.000000,2.878788,2.009174,0.151515,0.271795,0.018779,3.644172,...,0.066176,0.053571,0.000000,0.051282,0.000000,423.500000,1.363636,SMA06F_9,22,G
10,1.752000,0.023077,0.066667,0.005682,0.187879,0.064220,0.626263,0.148718,0.028169,0.128834,...,0.193627,0.062500,0.033537,0.000000,0.000000,658.682927,2.341463,SMA06F_10,41,H


In [6]:
# remove duplicated
test3_master = test3_master.loc[~test3_master.duplicated()]
# generate patient ID
test3_master['patient_ID'] = test3_master['obj_ID'].apply(lambda x: x.split('_')[0]).copy()
# re-order columns
columns = test3_master.columns.tolist()
columns[0], columns[362] = columns[362], columns[0]
columns[1], columns[365] = columns[365], columns[1]
columns[2], columns[364] = columns[364], columns[2]
test3_master_new = test3_master[columns]
test3_master_new

,obj_ID,patient_ID,class,Intensity_IntegratedIntensityEdge_CD38_rs,Intensity_IntegratedIntensityEdge_CD3_rs,Intensity_IntegratedIntensityEdge_CD45_rs,Intensity_IntegratedIntensityEdge_CD4_rs,Intensity_IntegratedIntensityEdge_CD5_rs,Intensity_IntegratedIntensityEdge_CD68_rs,Intensity_IntegratedIntensityEdge_CD8_rs,...,Intensity_UpperQuartileIntensity_PD1_rs,Intensity_UpperQuartileIntensity_PDL1_rs,Intensity_UpperQuartileIntensity_TBET_rs,Intensity_UpperQuartileIntensity_TCRB_rs,Location_Center_X,Location_Center_Y,Intensity_IntegratedIntensityEdge_CD138_rs,AreaShape_Area,Intensity_IntegratedIntensityEdge_CD27_rs,Intensity_IntegratedIntensityEdge_CD20_rs
1,SMA06F_1,SMA06F,G,0.000000,0.018182,0.018349,0.272727,0.015385,0.000000,0.012270,...,0.017857,0.000000,0.000000,0.000000,24.500000,1.285714,0.008000,14,0.025000,0.000000
2,SMA06F_2,SMA06F,B,0.255682,4.818182,6.128440,4.383838,3.374359,0.004695,0.184049,...,0.044643,0.036585,0.034188,0.004608,36.521739,3.144928,0.120000,69,1.808333,0.015385
3,SMA06F_3,SMA06F,B,0.187500,6.642424,10.055046,5.979798,2.651282,0.046948,0.190184,...,0.473214,0.012195,0.094017,0.004608,110.648148,5.055556,0.376000,108,9.591667,0.569231
4,SMA06F_4,SMA06F,B,0.096591,4.369697,3.587156,1.323232,1.266667,0.009390,0.024540,...,0.008929,0.000000,0.034188,0.000000,130.466667,3.746667,0.064000,75,0.258333,0.023077
5,SMA06F_5,SMA06F,B,0.011364,6.466667,2.229358,3.181818,8.225641,0.436620,0.006135,...,0.102679,0.018293,0.038462,0.200461,149.354545,6.881818,0.168000,110,0.433333,0.030769
6,SMA06F_6,SMA06F,H,0.017045,0.284848,0.275229,1.191919,0.333333,0.000000,0.184049,...,0.017857,0.048780,0.000000,0.004608,310.659574,2.851064,0.584000,94,0.458333,0.315385
7,SMA06F_7,SMA06F,H,0.017045,0.181818,0.137615,1.434343,0.076923,0.009390,0.061350,...,0.035714,0.024390,0.000000,0.000000,343.705263,6.473684,1.864000,95,0.325000,0.015385
8,SMA06F_8,SMA06F,D,0.068182,1.557576,2.073395,2.727273,0.230769,2.788732,0.036810,...,0.035714,0.036585,0.000000,0.000000,378.138889,5.097222,0.128000,72,0.183333,0.015385
9,SMA06F_9,SMA06F,G,0.000000,2.878788,2.009174,0.151515,0.271795,0.018779,3.644172,...,0.053571,0.000000,0.051282,0.000000,423.500000,1.363636,0.552000,22,0.058333,0.000000
10,SMA06F_10,SMA06F,H,0.005682,0.187879,0.064220,0.626263,0.148718,0.028169,0.128834,...,0.062500,0.033537,0.000000,0.000000,658.682927,2.341463,1.752000,41,0.066667,0.023077


In [7]:
# data imbalance analysis
test3_master_new['patient_ID'].value_counts(), test3_master_new['class'].value_counts()

(SMA19F    53773
 SMA37F    45614
 SMA40F    21363
 SMA06F    17459
 SMA54F    13718
 SMA60F    11566
 SMA21F     9476
 Name: patient_ID, dtype: int64, F    113129
 H     26684
 G     14324
 E      9796
 B      3899
 A      3025
 D      1391
 C       721
 Name: class, dtype: int64)

In [8]:
# subsample A groups
group_A = test3_master_new[test3_master_new['class'] == 'A']
group_A_choice = group_A
group_A_choice

,obj_ID,patient_ID,class,Intensity_IntegratedIntensityEdge_CD38_rs,Intensity_IntegratedIntensityEdge_CD3_rs,Intensity_IntegratedIntensityEdge_CD45_rs,Intensity_IntegratedIntensityEdge_CD4_rs,Intensity_IntegratedIntensityEdge_CD5_rs,Intensity_IntegratedIntensityEdge_CD68_rs,Intensity_IntegratedIntensityEdge_CD8_rs,...,Intensity_UpperQuartileIntensity_PD1_rs,Intensity_UpperQuartileIntensity_PDL1_rs,Intensity_UpperQuartileIntensity_TBET_rs,Intensity_UpperQuartileIntensity_TCRB_rs,Location_Center_X,Location_Center_Y,Intensity_IntegratedIntensityEdge_CD138_rs,AreaShape_Area,Intensity_IntegratedIntensityEdge_CD27_rs,Intensity_IntegratedIntensityEdge_CD20_rs
55,SMA06F_55,SMA06F,A,0.562500,5.654545,8.128440,1.101010,1.133333,3.544601,9.411043,...,0.017857,0.036585,0.034188,0.000000,2744.925466,3.366460,0.088000,161,1.041667,0.076923
72,SMA06F_72,SMA06F,A,0.000000,13.121212,3.412844,0.050505,0.158974,0.023474,6.490798,...,0.116071,0.000000,0.267094,0.069124,3760.303704,3.755556,0.000000,135,0.158333,0.215385
74,SMA06F_74,SMA06F,A,0.000000,8.496970,4.779817,0.474747,1.835897,0.000000,11.196319,...,0.116071,0.000000,0.136752,0.556452,3828.129412,2.964706,0.000000,85,1.191667,0.000000
75,SMA06F_75,SMA06F,A,0.153409,7.551515,5.302752,0.494949,0.907692,0.018779,7.987730,...,0.053571,0.000000,0.196581,0.000000,3895.500000,2.300000,0.056000,40,1.516667,0.023077
76,SMA06F_76,SMA06F,A,1.772727,7.072727,3.596330,0.101010,0.676923,0.051643,9.257669,...,0.125000,0.000000,0.239316,0.004608,3927.212121,2.666667,0.008000,66,2.241667,0.469231
78,SMA06F_78,SMA06F,A,7.596591,4.072727,4.834863,1.929293,0.584615,0.403756,10.073620,...,0.006696,0.060976,0.051282,0.210829,981.397790,12.060773,1.424000,181,8.525000,0.484615
108,SMA06F_108,SMA06F,A,0.340909,6.448485,5.532110,0.424242,1.061538,0.394366,12.233129,...,0.044643,0.024390,0.111111,0.009217,3614.991525,10.042373,0.024000,118,0.441667,0.076923
152,SMA06F_152,SMA06F,A,0.369318,11.672727,10.477064,0.606061,10.841026,0.145540,16.662577,...,0.000000,0.000000,0.119658,0.004608,3780.918182,14.400000,0.000000,110,4.083333,0.046154
156,SMA06F_156,SMA06F,A,0.062500,3.593939,5.853211,0.737374,0.574359,0.023474,6.049080,...,0.008929,0.024390,0.042735,0.110599,929.989130,13.478261,0.328000,92,0.408333,0.115385
168,SMA06F_168,SMA06F,A,0.909091,3.290909,3.091743,2.050505,0.507692,0.934272,4.674847,...,0.035714,0.012195,0.000000,0.110599,942.573913,16.643478,0.976000,115,1.208333,0.169231


In [9]:
# subsample B groups
group_B = test3_master_new[test3_master_new['class'] == 'B']
group_B_choice = group_B
group_B_choice

,obj_ID,patient_ID,class,Intensity_IntegratedIntensityEdge_CD38_rs,Intensity_IntegratedIntensityEdge_CD3_rs,Intensity_IntegratedIntensityEdge_CD45_rs,Intensity_IntegratedIntensityEdge_CD4_rs,Intensity_IntegratedIntensityEdge_CD5_rs,Intensity_IntegratedIntensityEdge_CD68_rs,Intensity_IntegratedIntensityEdge_CD8_rs,...,Intensity_UpperQuartileIntensity_PD1_rs,Intensity_UpperQuartileIntensity_PDL1_rs,Intensity_UpperQuartileIntensity_TBET_rs,Intensity_UpperQuartileIntensity_TCRB_rs,Location_Center_X,Location_Center_Y,Intensity_IntegratedIntensityEdge_CD138_rs,AreaShape_Area,Intensity_IntegratedIntensityEdge_CD27_rs,Intensity_IntegratedIntensityEdge_CD20_rs
2,SMA06F_2,SMA06F,B,0.255682,4.818182,6.128440,4.383838,3.374359,0.004695,0.184049,...,0.044643,0.036585,0.034188,0.004608,36.521739,3.144928,0.120000,69,1.808333,0.015385
3,SMA06F_3,SMA06F,B,0.187500,6.642424,10.055046,5.979798,2.651282,0.046948,0.190184,...,0.473214,0.012195,0.094017,0.004608,110.648148,5.055556,0.376000,108,9.591667,0.569231
4,SMA06F_4,SMA06F,B,0.096591,4.369697,3.587156,1.323232,1.266667,0.009390,0.024540,...,0.008929,0.000000,0.034188,0.000000,130.466667,3.746667,0.064000,75,0.258333,0.023077
5,SMA06F_5,SMA06F,B,0.011364,6.466667,2.229358,3.181818,8.225641,0.436620,0.006135,...,0.102679,0.018293,0.038462,0.200461,149.354545,6.881818,0.168000,110,0.433333,0.030769
68,SMA06F_68,SMA06F,B,0.193182,6.903030,2.807340,3.252525,3.041026,0.000000,0.079755,...,0.062500,0.036585,0.188034,0.009217,3587.090000,3.570000,0.008000,100,1.775000,0.000000
167,SMA06F_167,SMA06F,B,0.045455,11.418182,9.100918,4.232323,1.184615,1.826291,2.595092,...,0.151786,0.036585,0.017094,0.000000,390.200000,16.207143,0.600000,140,0.350000,0.046154
180,SMA06F_180,SMA06F,B,0.164773,6.048485,8.348624,3.727273,2.374359,0.154930,0.159509,...,0.321429,0.000000,0.017094,0.009217,108.509091,18.190909,0.392000,110,8.741667,0.053846
197,SMA06F_197,SMA06F,B,0.403409,3.266667,4.073395,3.161616,1.179487,11.610329,0.319018,...,0.044643,0.036585,0.017094,0.004608,378.338235,18.397059,0.472000,68,2.491667,0.000000
222,SMA06F_222,SMA06F,B,0.125000,18.048485,10.302753,9.353535,7.328205,0.009390,3.411043,...,0.053571,0.000000,0.213675,0.046083,3819.189320,24.640777,0.024000,206,15.233334,0.146154
283,SMA06F_283,SMA06F,B,0.357955,5.830303,6.348624,3.090909,2.241026,0.164319,0.042945,...,0.071429,0.024390,0.051282,0.000000,57.788079,31.589404,0.280000,151,4.483333,0.007692


In [10]:
# subsample C groups
group_C = test3_master_new[test3_master_new['class'] == 'C']
group_C_choice = group_C
group_C_choice

,obj_ID,patient_ID,class,Intensity_IntegratedIntensityEdge_CD38_rs,Intensity_IntegratedIntensityEdge_CD3_rs,Intensity_IntegratedIntensityEdge_CD45_rs,Intensity_IntegratedIntensityEdge_CD4_rs,Intensity_IntegratedIntensityEdge_CD5_rs,Intensity_IntegratedIntensityEdge_CD68_rs,Intensity_IntegratedIntensityEdge_CD8_rs,...,Intensity_UpperQuartileIntensity_PD1_rs,Intensity_UpperQuartileIntensity_PDL1_rs,Intensity_UpperQuartileIntensity_TBET_rs,Intensity_UpperQuartileIntensity_TCRB_rs,Location_Center_X,Location_Center_Y,Intensity_IntegratedIntensityEdge_CD138_rs,AreaShape_Area,Intensity_IntegratedIntensityEdge_CD27_rs,Intensity_IntegratedIntensityEdge_CD20_rs
61,SMA06F_61,SMA06F,C,0.823864,0.739394,1.449541,2.141414,0.656410,0.666667,1.159509,...,0.084821,0.085366,0.119658,0.069124,2998.448980,1.877551,0.752000,49,2.083333,1.415385
904,SMA06F_904,SMA06F,C,1.386364,1.175758,5.495413,0.929293,0.615385,2.291080,1.969325,...,0.044643,0.012195,0.085470,0.055300,3184.826923,89.205128,0.432000,156,8.766667,2.492308
1173,SMA06F_1173,SMA06F,C,0.477273,0.357576,0.899083,0.757576,0.256410,0.502347,0.546012,...,0.000000,0.009146,0.059829,0.050691,3234.969697,113.575758,0.400000,33,1.808333,1.076923
2730,SMA06F_2730,SMA06F,C,0.380682,1.236364,3.119266,0.565657,0.066667,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,42.174757,266.757282,0.000000,103,0.250000,7.923077
2784,SMA06F_2784,SMA06F,C,0.113636,0.278788,3.293578,0.323232,0.200000,0.051643,0.006135,...,0.026786,0.000000,0.000000,0.000000,282.198630,272.061644,0.008000,146,1.183333,9.376923
3882,SMA06F_3882,SMA06F,C,0.613636,0.727273,1.146789,1.515152,0.471795,0.629108,0.803681,...,0.125000,0.121951,0.115385,0.071429,3044.185185,373.740741,0.872000,54,1.425000,1.030769
3915,SMA06F_3915,SMA06F,C,0.062500,0.187879,1.981651,0.414141,0.092308,0.004695,0.024540,...,0.017857,0.121951,0.000000,0.000000,1009.800000,377.980000,0.016000,50,0.641667,8.769230
5468,SMA06F_5468,SMA06F,C,0.159091,0.660606,2.972477,1.353535,0.046154,0.863850,0.521472,...,0.020089,0.039634,0.000000,0.000000,894.666667,529.528455,0.208000,123,0.816667,15.276923
6054,SMA06F_6054,SMA06F,C,0.920455,0.933333,1.302752,1.686869,0.800000,0.671362,1.104294,...,0.098214,0.243902,0.119658,0.082949,1650.138889,582.611111,0.760000,36,1.208333,1.207692
7527,SMA06F_7527,SMA06F,C,0.000000,0.000000,2.752294,0.060606,0.005128,0.000000,0.012270,...,0.000000,0.000000,0.000000,0.000000,1910.650485,727.271845,0.000000,103,0.175000,4.192307


In [11]:
# subsample D groups
group_D = test3_master_new[test3_master_new['class'] == 'D']
group_D_choice = group_D
group_D_choice

,obj_ID,patient_ID,class,Intensity_IntegratedIntensityEdge_CD38_rs,Intensity_IntegratedIntensityEdge_CD3_rs,Intensity_IntegratedIntensityEdge_CD45_rs,Intensity_IntegratedIntensityEdge_CD4_rs,Intensity_IntegratedIntensityEdge_CD5_rs,Intensity_IntegratedIntensityEdge_CD68_rs,Intensity_IntegratedIntensityEdge_CD8_rs,...,Intensity_UpperQuartileIntensity_PD1_rs,Intensity_UpperQuartileIntensity_PDL1_rs,Intensity_UpperQuartileIntensity_TBET_rs,Intensity_UpperQuartileIntensity_TCRB_rs,Location_Center_X,Location_Center_Y,Intensity_IntegratedIntensityEdge_CD138_rs,AreaShape_Area,Intensity_IntegratedIntensityEdge_CD27_rs,Intensity_IntegratedIntensityEdge_CD20_rs
8,SMA06F_8,SMA06F,D,0.068182,1.557576,2.073395,2.727273,0.230769,2.788732,0.036810,...,0.035714,0.036585,0.000000,0.000000,378.138889,5.097222,0.128000,72,0.183333,0.015385
66,SMA06F_66,SMA06F,D,0.562500,0.787879,3.091743,0.868687,0.333333,0.929577,0.312883,...,0.026786,0.024390,0.034188,0.009217,3356.853659,4.764228,0.152000,123,1.291667,0.300000
140,SMA06F_140,SMA06F,D,0.363636,0.412121,1.183486,1.252525,0.184615,1.591549,0.441718,...,0.089286,0.036585,0.000000,0.000000,3177.622951,12.049180,0.576000,61,1.741667,0.907692
149,SMA06F_149,SMA06F,D,1.289773,0.448485,5.935780,1.212121,0.276923,4.657277,0.134969,...,0.049107,0.091463,0.047009,0.009217,3369.088235,12.323529,0.672000,34,3.075000,0.376923
202,SMA06F_202,SMA06F,D,0.573864,2.387879,5.899083,4.686869,0.348718,1.779343,0.159509,...,0.044643,0.024390,0.025641,0.000000,3734.750000,18.594828,0.040000,116,1.133333,0.100000
221,SMA06F_221,SMA06F,D,0.687500,0.472727,3.064220,1.070707,1.066667,3.187793,0.214724,...,0.133929,0.201220,0.094017,0.032258,3375.666667,18.866667,0.360000,45,1.833333,0.315385
241,SMA06F_241,SMA06F,D,0.642045,0.181818,1.596330,0.676768,0.128205,11.896714,0.110429,...,0.000000,0.012195,0.000000,0.000000,3158.000000,22.581818,0.216000,55,0.875000,0.392308
254,SMA06F_254,SMA06F,D,0.187500,0.818182,4.027523,3.040404,0.348718,3.380282,0.030675,...,0.013393,0.036585,0.025641,0.004608,40.440299,29.634328,0.128000,134,1.008333,0.038462
318,SMA06F_318,SMA06F,D,3.613636,0.212121,2.137615,1.848485,0.051282,2.563380,0.460123,...,0.044643,0.060976,0.025641,0.009217,1052.840909,29.659091,2.352000,44,4.508333,0.323077
430,SMA06F_430,SMA06F,D,0.051136,0.103030,3.990826,0.929293,0.476923,2.244131,0.085890,...,0.000000,0.012195,0.000000,0.000000,844.000000,43.490323,0.064000,155,0.266667,0.000000


In [21]:
# subsample E groups
group_E = test3_master_new[test3_master_new['class'] == 'E']
group_E['patient_ID'].value_counts()

SMA37F    3014
SMA19F    1942
SMA40F    1905
SMA60F    1592
SMA54F     529
SMA06F     453
SMA21F     361
Name: patient_ID, dtype: int64

In [12]:
# balanced sample per patient for E group
group_E_SMA21F = group_E[group_E['patient_ID'] == 'SMA21F']
group_E_SMA54F = group_E[group_E['patient_ID'] == 'SMA54F']
group_E_SMA06F = group_E[group_E['patient_ID'] == 'SMA06F']

group_E_SMA60F = group_E[group_E['patient_ID'] == 'SMA60F']
group_E_SMA40F = group_E[group_E['patient_ID'] == 'SMA40F']
group_E_SMA19F = group_E[group_E['patient_ID'] == 'SMA19F']
group_E_SMA37F = group_E[group_E['patient_ID'] == 'SMA37F']

# sample 900 cells from each patient
group_E_SMA60F = group_E_SMA60F.iloc[np.random.choice(group_E_SMA60F.shape[0], 900, replace=False)]
group_E_SMA40F = group_E_SMA40F.iloc[np.random.choice(group_E_SMA40F.shape[0], 900, replace=False)]
group_E_SMA19F = group_E_SMA19F.iloc[np.random.choice(group_E_SMA19F.shape[0], 900, replace=False)]
group_E_SMA37F = group_E_SMA37F.iloc[np.random.choice(group_E_SMA37F.shape[0], 900, replace=False)]

group_E_choice = group_E_SMA21F.append(group_E_SMA54F, ignore_index=True)\
                               .append(group_E_SMA06F, ignore_index=True)\
                               .append(group_E_SMA60F, ignore_index=True)\
                               .append(group_E_SMA40F, ignore_index=True)\
                               .append(group_E_SMA19F, ignore_index=True)\
                               .append(group_E_SMA37F, ignore_index=True)
group_E_choice['patient_ID'].value_counts()

SMA37F    900
SMA19F    900
SMA60F    900
SMA40F    900
SMA54F    529
SMA06F    453
SMA21F    361
Name: patient_ID, dtype: int64

In [13]:
# subsample F groups
group_F = test3_master_new[test3_master_new['class'] == 'F']
group_F['patient_ID'].value_counts()

SMA19F    39852
SMA37F    30877
SMA40F    10713
SMA06F    10170
SMA54F     8152
SMA21F     7702
SMA60F     5663
Name: patient_ID, dtype: int64

In [14]:
# balanced sample per patient for F group
group_F_SMA21F = group_F[group_F['patient_ID'] == 'SMA21F']
group_F_SMA54F = group_F[group_F['patient_ID'] == 'SMA54F']
group_F_SMA06F = group_F[group_F['patient_ID'] == 'SMA06F']
group_F_SMA60F = group_F[group_F['patient_ID'] == 'SMA60F']
group_F_SMA40F = group_F[group_F['patient_ID'] == 'SMA40F']
group_F_SMA19F = group_F[group_F['patient_ID'] == 'SMA19F']
group_F_SMA37F = group_F[group_F['patient_ID'] == 'SMA37F']

# sample 650 cells from each patient
group_F_SMA21F = group_F_SMA21F.iloc[np.random.choice(group_F_SMA21F.shape[0], 650, replace=False)]
group_F_SMA54F = group_F_SMA54F.iloc[np.random.choice(group_F_SMA54F.shape[0], 650, replace=False)]
group_F_SMA06F = group_F_SMA06F.iloc[np.random.choice(group_F_SMA06F.shape[0], 650, replace=False)]

group_F_SMA60F = group_F_SMA60F.iloc[np.random.choice(group_F_SMA60F.shape[0], 650, replace=False)]
group_F_SMA40F = group_F_SMA40F.iloc[np.random.choice(group_F_SMA40F.shape[0], 650, replace=False)]
group_F_SMA19F = group_F_SMA19F.iloc[np.random.choice(group_F_SMA19F.shape[0], 650, replace=False)]
group_F_SMA37F = group_F_SMA37F.iloc[np.random.choice(group_F_SMA37F.shape[0], 650, replace=False)]

group_F_choice = group_F_SMA21F.append(group_F_SMA54F, ignore_index=True)\
                               .append(group_F_SMA06F, ignore_index=True)\
                               .append(group_F_SMA60F, ignore_index=True)\
                               .append(group_F_SMA40F, ignore_index=True)\
                               .append(group_F_SMA19F, ignore_index=True)\
                               .append(group_F_SMA37F, ignore_index=True)
group_F_choice['patient_ID'].value_counts()

SMA54F    650
SMA37F    650
SMA06F    650
SMA19F    650
SMA60F    650
SMA21F    650
SMA40F    650
Name: patient_ID, dtype: int64

In [15]:
# subsample G groups
group_G = test3_master_new[test3_master_new['class'] == 'G']
group_G['patient_ID'].value_counts()

SMA19F    4582
SMA37F    2725
SMA06F    2497
SMA40F    1768
SMA21F    1124
SMA54F     933
SMA60F     695
Name: patient_ID, dtype: int64

In [16]:
# balanced sample per patient for G group
group_G_SMA21F = group_G[group_G['patient_ID'] == 'SMA21F']
group_G_SMA54F = group_G[group_G['patient_ID'] == 'SMA54F']
group_G_SMA06F = group_G[group_G['patient_ID'] == 'SMA06F']
group_G_SMA60F = group_G[group_G['patient_ID'] == 'SMA60F']
group_G_SMA40F = group_G[group_G['patient_ID'] == 'SMA40F']
group_G_SMA19F = group_G[group_G['patient_ID'] == 'SMA19F']
group_G_SMA37F = group_G[group_G['patient_ID'] == 'SMA37F']

# sample 650 cells from each patient
group_G_SMA21F = group_G_SMA21F.iloc[np.random.choice(group_G_SMA21F.shape[0], 650, replace=False)]
group_G_SMA54F = group_G_SMA54F.iloc[np.random.choice(group_G_SMA54F.shape[0], 650, replace=False)]
group_G_SMA06F = group_G_SMA06F.iloc[np.random.choice(group_G_SMA06F.shape[0], 650, replace=False)]

group_G_SMA60F = group_G_SMA60F.iloc[np.random.choice(group_G_SMA60F.shape[0], 650, replace=False)]
group_G_SMA40F = group_G_SMA40F.iloc[np.random.choice(group_G_SMA40F.shape[0], 650, replace=False)]
group_G_SMA19F = group_G_SMA19F.iloc[np.random.choice(group_G_SMA19F.shape[0], 650, replace=False)]
group_G_SMA37F = group_G_SMA37F.iloc[np.random.choice(group_G_SMA37F.shape[0], 650, replace=False)]

group_G_choice = group_G_SMA21F.append(group_G_SMA54F, ignore_index=True)\
                               .append(group_G_SMA06F, ignore_index=True)\
                               .append(group_G_SMA60F, ignore_index=True)\
                               .append(group_G_SMA40F, ignore_index=True)\
                               .append(group_G_SMA19F, ignore_index=True)\
                               .append(group_G_SMA37F, ignore_index=True)
group_G_choice['patient_ID'].value_counts()

SMA54F    650
SMA37F    650
SMA06F    650
SMA19F    650
SMA60F    650
SMA21F    650
SMA40F    650
Name: patient_ID, dtype: int64

In [17]:
# subsample H groups
group_H = test3_master_new[test3_master_new['class'] == 'H']
group_H['patient_ID'].value_counts()

SMA37F    7497
SMA19F    6531
SMA40F    4190
SMA54F    3397
SMA06F    2567
SMA60F    2476
SMA21F      26
Name: patient_ID, dtype: int64

In [18]:
# balanced sample per patient for H group
group_H_SMA21F = group_H[group_H['patient_ID'] == 'SMA21F']
group_H_SMA54F = group_H[group_H['patient_ID'] == 'SMA54F']
group_H_SMA06F = group_H[group_H['patient_ID'] == 'SMA06F']
group_H_SMA60F = group_H[group_H['patient_ID'] == 'SMA60F']
group_H_SMA40F = group_H[group_H['patient_ID'] == 'SMA40F']
group_H_SMA19F = group_H[group_H['patient_ID'] == 'SMA19F']
group_H_SMA37F = group_H[group_H['patient_ID'] == 'SMA37F']

# sample 650s cell from each patient
#group_H_SMA21F = group_H_SMA21F.iloc[np.random.choice(group_H_SMA21F.shape[0], 650, replace=False)]
group_H_SMA54F = group_H_SMA54F.iloc[np.random.choice(group_H_SMA54F.shape[0], 650, replace=False)]
group_H_SMA06F = group_H_SMA06F.iloc[np.random.choice(group_H_SMA06F.shape[0], 650, replace=False)]

group_H_SMA60F = group_H_SMA60F.iloc[np.random.choice(group_H_SMA60F.shape[0], 650, replace=False)]
group_H_SMA40F = group_H_SMA40F.iloc[np.random.choice(group_H_SMA40F.shape[0], 650, replace=False)]
group_H_SMA19F = group_H_SMA19F.iloc[np.random.choice(group_H_SMA19F.shape[0], 650, replace=False)]
group_H_SMA37F = group_H_SMA37F.iloc[np.random.choice(group_H_SMA37F.shape[0], 650, replace=False)]

group_H_choice = group_H_SMA21F.append(group_H_SMA54F, ignore_index=True)\
                               .append(group_H_SMA06F, ignore_index=True)\
                               .append(group_H_SMA60F, ignore_index=True)\
                               .append(group_H_SMA40F, ignore_index=True)\
                               .append(group_H_SMA19F, ignore_index=True)\
                               .append(group_H_SMA37F, ignore_index=True)
group_H_choice['patient_ID'].value_counts()

SMA54F    650
SMA37F    650
SMA06F    650
SMA19F    650
SMA60F    650
SMA40F    650
SMA21F     26
Name: patient_ID, dtype: int64

In [19]:
## combine each group together and make ml dataset ready
test3_master_ready = group_A_choice.append(group_B_choice, ignore_index=True)\
                                   .append(group_C_choice, ignore_index=True)\
                                   .append(group_D_choice, ignore_index=True)\
                                   .append(group_E_choice, ignore_index=True)\
                                   .append(group_F_choice, ignore_index=True)\
                                   .append(group_G_choice, ignore_index=True)\
                                   .append(group_H_choice, ignore_index=True)
test3_master_ready

,obj_ID,patient_ID,class,Intensity_IntegratedIntensityEdge_CD38_rs,Intensity_IntegratedIntensityEdge_CD3_rs,Intensity_IntegratedIntensityEdge_CD45_rs,Intensity_IntegratedIntensityEdge_CD4_rs,Intensity_IntegratedIntensityEdge_CD5_rs,Intensity_IntegratedIntensityEdge_CD68_rs,Intensity_IntegratedIntensityEdge_CD8_rs,...,Intensity_UpperQuartileIntensity_PD1_rs,Intensity_UpperQuartileIntensity_PDL1_rs,Intensity_UpperQuartileIntensity_TBET_rs,Intensity_UpperQuartileIntensity_TCRB_rs,Location_Center_X,Location_Center_Y,Intensity_IntegratedIntensityEdge_CD138_rs,AreaShape_Area,Intensity_IntegratedIntensityEdge_CD27_rs,Intensity_IntegratedIntensityEdge_CD20_rs
0,SMA06F_55,SMA06F,A,0.562500,5.654545,8.128440,1.101010,1.133333,3.544601,9.411043,...,0.017857,0.036585,0.034188,0.000000,2744.925466,3.366460,0.088000,161,1.041667,0.076923
1,SMA06F_72,SMA06F,A,0.000000,13.121212,3.412844,0.050505,0.158974,0.023474,6.490798,...,0.116071,0.000000,0.267094,0.069124,3760.303704,3.755556,0.000000,135,0.158333,0.215385
2,SMA06F_74,SMA06F,A,0.000000,8.496970,4.779817,0.474747,1.835897,0.000000,11.196319,...,0.116071,0.000000,0.136752,0.556452,3828.129412,2.964706,0.000000,85,1.191667,0.000000
3,SMA06F_75,SMA06F,A,0.153409,7.551515,5.302752,0.494949,0.907692,0.018779,7.987730,...,0.053571,0.000000,0.196581,0.000000,3895.500000,2.300000,0.056000,40,1.516667,0.023077
4,SMA06F_76,SMA06F,A,1.772727,7.072727,3.596330,0.101010,0.676923,0.051643,9.257669,...,0.125000,0.000000,0.239316,0.004608,3927.212121,2.666667,0.008000,66,2.241667,0.469231
5,SMA06F_78,SMA06F,A,7.596591,4.072727,4.834863,1.929293,0.584615,0.403756,10.073620,...,0.006696,0.060976,0.051282,0.210829,981.397790,12.060773,1.424000,181,8.525000,0.484615
6,SMA06F_108,SMA06F,A,0.340909,6.448485,5.532110,0.424242,1.061538,0.394366,12.233129,...,0.044643,0.024390,0.111111,0.009217,3614.991525,10.042373,0.024000,118,0.441667,0.076923
7,SMA06F_152,SMA06F,A,0.369318,11.672727,10.477064,0.606061,10.841026,0.145540,16.662577,...,0.000000,0.000000,0.119658,0.004608,3780.918182,14.400000,0.000000,110,4.083333,0.046154
8,SMA06F_156,SMA06F,A,0.062500,3.593939,5.853211,0.737374,0.574359,0.023474,6.049080,...,0.008929,0.024390,0.042735,0.110599,929.989130,13.478261,0.328000,92,0.408333,0.115385
9,SMA06F_168,SMA06F,A,0.909091,3.290909,3.091743,2.050505,0.507692,0.934272,4.674847,...,0.035714,0.012195,0.000000,0.110599,942.573913,16.643478,0.976000,115,1.208333,0.169231


In [22]:
test3_master_ready.duplicated().any()

False

In [23]:
test3_master_ready['class'].value_counts()

E    4943
G    4550
F    4550
H    3926
B    3899
A    3025
D    1391
C     721
Name: class, dtype: int64

In [24]:
test3_master_ready['patient_ID'].value_counts()

SMA40F    5637
SMA37F    4351
SMA06F    4175
SMA60F    3990
SMA19F    3716
SMA54F    3186
SMA21F    1950
Name: patient_ID, dtype: int64

In [25]:
test3_master_ready.to_csv('./mIHC/test3/test3_master_ready_1.tsv', sep='\t', header=True, index=False)